In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import os
import csv
import copy
import json
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence, Tuple, List
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import torch
import transformers
import sklearn
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
import numpy as np
from torch.utils.data import Dataset
from scipy.special import softmax
from peftnew import (
    LoraConfig,
    AdaLoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    AdaLoraModel
)

from transformers import Trainer

/common/zhanh/anaconda3/envs/dna_peft/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CustomTrainer(Trainer):

    def training_step(self, model, inputs):
        # Call original training step
        outputs = super().training_step(model, inputs)

        if self.state.global_step % 50 == 0:
            # Check gradients after backward pass
            #self.check_gradients(model)

            # Update and allocate
            model.update_and_allocate(self.state.global_step)

        return outputs

    def check_gradients(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                if param.grad is None:
                    print(f"No gradient for {name}")
                else:
                    gradient_norm = param.grad.norm().item()
                    print(f"Gradient for {name} exists with norm: {gradient_norm}")


In [4]:
from transformers import TrainerCallback, TrainerControl, TrainerState, TrainingArguments

class EvalAndSaveCallback(TrainerCallback):
    def __init__(self, model, tokenizer, trainer):
        self.trainer = trainer
        self.model = model
        self.tokenizer = tokenizer
    
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 500 == 0:
            results = self.trainer.evaluate()
            results_file = os.path.join(args.output_dir, f"results_step_{state.global_step}.json")
            output_file_path = os.path.join(args.output_dir, "results.csv")
            with open(results_file, "w") as f:
                json.dump(results, f)
            with open(output_file_path, "a", newline='') as csv_file:
                writer = csv.writer(csv_file)
        
                # Write keys (dictionary's keys) to the first row
                if state.global_step == 500:
                    writer.writerow(results.keys())

                # Write values in the subsequent rows
                writer.writerow(results.values())



In [5]:
@dataclass
class ModelArguments:
    #model_name_or_path: Optional[str] = field(default="zhihan1996/DNABERT-2-117M")
    model_name_or_path: Optional[str] = field(default="facebook/opt-125m")
    #model_name_or_path: Optional[str] = field(default="decapoda-research/llama-7b-hf")
    #model_name_or_path: Optional[str] = field(default="microsoft/MiniLM-L12-H384-uncased")
    use_lora: bool = field(default=True, metadata={"help": "whether to use LoRA"})
    lora_r: int = field(default=8, metadata={"help": "hidden dimension for LoRA"})
    lora_alpha: int = field(default=32, metadata={"help": "alpha for LoRA"})
    lora_dropout: float = field(default=0.05, metadata={"help": "dropout rate for LoRA"})
    lora_target_modules: str = field(default="k_proj,q_proj,v_proj,fc1,fc2,output_proj", metadata={"help": "where to perform LoRA"})
    #lora_target_modules: str = field(default="Wqkv,dense,mlp.wo", metadata={"help": "where to perform LoRA"})
    #lora_target_modules: str = field(default="query,key,value", metadata={"help": "where to perform LoRA"})

In [6]:
@dataclass
class DataArguments:
    data_path: str = field(default="/common/zhangz2lab/zhanh/GUE/prom/prom_300_all", metadata={"help": "Path to the training data."})
    kmer: int = field(default=-1, metadata={"help": "k-mer for input sequence. -1 means not using k-mer."})

In [7]:
import logging
logging.basicConfig(level=logging.ERROR)

@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    run_name: str = field(default="run")
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(default=512, metadata={"help": "Maximum sequence length."})
    gradient_accumulation_steps: int = field(default=1)
    per_device_train_batch_size: int = field(default=8)
    per_device_eval_batch_size: int = field(default=16)
    num_train_epochs: int = field(default=5)
    fp16: bool = field(default=False)
    logging_steps: int = field(default=1000)
    save_steps: int = field(default=500)
    eval_steps: int = field(default=500)
    evaluation_strategy: str = field(default="steps")
    load_best_model_at_end: bool = field(default=True)     # load the best model when finished training (default metric is loss)
    metric_for_best_model: str = field(default="matthews_correlation") # the metric to use to compare models
    greater_is_better: bool = field(default=True)           # whether the `metric_for_best_model` should be maximized or not
    logging_strategy: str = field(default="steps")  # Log every "steps"
    logging_steps: int = field(default=100)  # Log every 100 steps
    warmup_ratio: int = field(default=0.1)
    weight_decay: float = field(default=5e-3)
    learning_rate: float = field(default=3e-5)
    lr_scheduler_type: str = field(default='linear')
    save_total_limit: int = field(default=10)
    load_best_model_at_end: bool = field(default=True)
    output_dir: str = field(default="/common/zhangz2lab/zhanh/output_budget_0918")
    find_unused_parameters: bool = field(default=False)
    checkpointing: bool = field(default=False)
    dataloader_pin_memory: bool = field(default=False)
    eval_and_save_results: bool = field(default=True)
    save_model: bool = field(default=False)
    seed: int = field(default=42)
    logging_first_step: bool = field(default=True)
    early_stopping_patience: int = field(default = 5)  # number of evaluations without improvement to wait
    early_stopping_threshold: float = field(default = 1e-3)  # threshold for an improvement

In [8]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    #state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        #cpu_state_dict = {key: value.cpu() for key, value in state_dict.items()}
        #del state_dict
        #trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa
        checkpoint_dir = os.path.join(output_dir, f"checkpoint-{trainer.state.global_step}")
        trainer.model.save_pretrained(checkpoint_dir)
        trainer.model.config.save_pretrained(checkpoint_dir) 

In [9]:
def get_alter_of_dna_sequence(sequence: str):
    MAP = {"A": "T", "T": "A", "C": "G", "G": "C"}
    # return "".join([MAP[c] for c in reversed(sequence)])
    return "".join([MAP[c] for c in sequence])

"""
Transform a dna sequence to k-mer string
"""
def generate_kmer_str(sequence: str, k: int) -> str:
    """Generate k-mer string from DNA sequence."""
    return " ".join([sequence[i:i+k] for i in range(len(sequence) - k + 1)])


"""
Load or generate k-mer string for each DNA sequence. The generated k-mer string will be saved to the same directory as the original data with the same name but with a suffix of "_{k}mer".
"""
def load_or_generate_kmer(data_path: str, texts: List[str], k: int) -> List[str]:
    """Load or generate k-mer string for each DNA sequence."""
    kmer_path = data_path.replace(".csv", f"_{k}mer.json")
    print(kmer_path)
    if os.path.exists(kmer_path):
        logging.warning(f"Loading k-mer from {kmer_path}...")
        with open(kmer_path, "r") as f:
            kmer = json.load(f)
    else:        
        logging.warning(f"Generating k-mer...")
        kmer = [generate_kmer_str(text, k) for text in texts]
        with open(kmer_path, "w") as f:
            logging.warning(f"Saving k-mer to {kmer_path}...")
            json.dump(kmer, f)
        
    return kmer

In [10]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, 
                 data_path: str, 
                 tokenizer: transformers.PreTrainedTokenizer, 
                 kmer: int = -1):

        super(SupervisedDataset, self).__init__()

        # load data from the disk
        with open(data_path, "r") as f:
            data = list(csv.reader(f))[1:]
        if len(data[0]) == 2:
            # data is in the format of [text, label]
            logging.warning("Perform single sequence classification...")
            texts = [d[0] for d in data]
            labels = [int(d[1]) for d in data]
        elif len(data[0]) == 3:
            # data is in the format of [text1, text2, label]
            logging.warning("Perform sequence-pair classification...")
            texts = [[d[0], d[1]] for d in data]
            labels = [int(d[2]) for d in data]
        else:
            raise ValueError("Data format not supported.")
        
        if kmer != -1:
            # only write file on the first process
            #if torch.distributed.get_rank() not in [0, -1]:
            #    torch.distributed.barrier()

            logging.warning(f"Using {kmer}-mer as input...")
            texts = load_or_generate_kmer(data_path, texts, kmer)

            #if torch.distributed.get_rank() == 0:
            #    torch.distributed.barrier()

        output = tokenizer(
            texts,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )

        self.input_ids = output["input_ids"]
        self.attention_mask = output["attention_mask"]
        self.labels = labels
        self.num_labels = len(set(labels))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [11]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.Tensor(labels).long()
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [12]:
def calculate_metric_with_sklearn(logits: np.ndarray, labels: np.ndarray):
    if logits.ndim == 3:
        # Reshape logits to 2D if needed
        logits = logits.reshape(-1, logits.shape[-1])
    predictions = np.argmax(logits, axis=-1)
    valid_mask = labels != -100  # Exclude padding tokens (assuming -100 is the padding token ID)
    valid_predictions = predictions[valid_mask]
    valid_labels = labels[valid_mask]
    # Compute probabilities from logits
    probabilities = softmax(logits, axis=-1)

    # Extract the probabilities corresponding to the positive class
    valid_scores = probabilities[valid_mask, 1]  # assuming the second column is the positive class
    return {
        "accuracy": sklearn.metrics.accuracy_score(valid_labels, valid_predictions),
        "f1": sklearn.metrics.f1_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
        "matthews_correlation": sklearn.metrics.matthews_corrcoef(
            valid_labels, valid_predictions
        ),
        "precision": sklearn.metrics.precision_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
        "recall": sklearn.metrics.recall_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
        "pr_auc": average_precision_score(valid_labels, valid_scores),
        "roc_auc": roc_auc_score(valid_labels, valid_scores),
        "brier_score": brier_score_loss(valid_labels, valid_scores)
    }

In [13]:
"""
Compute metrics used for huggingface trainer.
""" 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):  # Unpack logits if it's a tuple
        logits = logits[0]
    return calculate_metric_with_sklearn(logits, labels)

In [14]:
def train():
    #parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    #print(parser)
    #model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments,))
    model_args, data_args, training_args, remaining = parser.parse_args_into_dataclasses(return_remaining_strings=True)

    # load tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=True,
        trust_remote_code=True,
    )
    

    if "InstaDeepAI" in model_args.model_name_or_path:
        tokenizer.eos_token = tokenizer.pad_token

    # define datasets and data collator
    train_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                      data_path=os.path.join(data_args.data_path, "train.csv"), 
                                      kmer=data_args.kmer)
    val_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                     data_path=os.path.join(data_args.data_path, "dev.csv"), 
                                     kmer=data_args.kmer)
    test_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                     data_path=os.path.join(data_args.data_path, "test.csv"), 
                                     kmer=data_args.kmer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

    #config = transformers.AutoConfig.from_pretrained(model_args.model_name_or_path, trust_remote_code=True)
    # load model
    model = transformers.AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        #config = config,
        cache_dir=training_args.cache_dir,
        num_labels=train_dataset.num_labels,
        trust_remote_code=True
    )
    #module_names = [name for name, _ in model.named_modules()]
    #print([(n, type(m)) for n, m in model.named_modules()])
    module_names_and_types = [(n, type(m)) for n, m in model.named_modules()]
    module = ",".join(n for n, _ in module_names_and_types)
    target = list(module.split(","))
    print(module)
    

    # Print the list
    #print(module_names)
    
    # Get the names of the layers in the base_model

    #for layer_idx, layer in enumerate(model.base_model.encoder.layer):
        #print(f"Layer {layer_idx}:")
    
        # Get the self-attention layer
        #self_attention_layer = layer.attention.self
    
        # Print the names of the sub-components
        #for name, module in self_attention_layer.named_children():
            #print(f"  {name}")

    # configure LoRA
    #model_args.lora_target_modules = r"bert\.encoder\.layer\.\d+\.mlp\.wo" 
    if model_args.use_lora:
        lora_config = AdaLoraConfig(
            r = model_args.lora_r,
            init_r = 8,
            target_r = 4,
            tinit=200,
            tfinal=200,
            total_step=7485,
            deltaT=5000,
            #target_modules=list(r"bert\.encoder\.layer\.\d+\.mlp\.wo"),
            lora_alpha=model_args.lora_alpha,
            target_modules=list(model_args.lora_target_modules.split(",")),
            #target_modules = target[1:],
            lora_dropout=model_args.lora_dropout,
            bias="none",
            task_type="SEQ_CLS",
            inference_mode=False,
            #peft_type="ADALORA",
        )
        print(list(model_args.lora_target_modules.split(",")))
        model = get_peft_model(model, lora_config)
        #model = AdaLoraModel(model, lora_config, "default")
        model.print_trainable_parameters()
        
    

    # define trainer
    trainer = CustomTrainer(model=model,
                                   tokenizer=tokenizer,
                                   args=training_args,
                                   compute_metrics=compute_metrics,
                                   train_dataset=train_dataset,
                                   eval_dataset=test_dataset,
                                   data_collator=data_collator,
                                  )
    
    callback = EvalAndSaveCallback(model, tokenizer, trainer)
    trainer.add_callback(callback)


    
    #results = trainer.evaluate()

    # Print or save these results if you want
    #print("Initial evaluation results:", results)
    
    trainer.train()

    if training_args.save_model:
        trainer.save_state()
        safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)

    # get the evaluation results from trainer
    if training_args.eval_and_save_results:
        results_path = os.path.join(training_args.output_dir, "results", training_args.run_name)
        print(results_path)
        results = trainer.evaluate(eval_dataset=test_dataset)
        os.makedirs(results_path, exist_ok=True)
        with open(os.path.join(results_path, "eval_results.json"), "w") as f:
            json.dump(results, f)
    
    # Get all checkpoint directories
#     if training_args.eval_and_save_results:
#     # Get list of all checkpoints
#         checkpoints = [dir_name for dir_name in os.listdir(training_args.output_dir) 
#                    if 'checkpoint' in dir_name]
    
#         for checkpoint in checkpoints:
#             checkpoint_path = os.path.join(training_args.output_dir, checkpoint)
        
#         # Load model from checkpoint
#             model_checkpoint = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_path,trust_remote_code=True)
#             trainer.model = model_checkpoint  # Update trainer's model
#             print(model.device)
#             print(next(model.parameters()).device)
#             for key, value in inputs.items():
#                 if isinstance(value, torch.Tensor):
#                     print(f"{key} is on {value.device}")
#             #print(labels.device)
#             device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#             input_ids = input_ids.to(device)
#             token_type_ids = token_type_ids.to(device)
#             attention_mask = attention_mask.to(device)

#             results = trainer.evaluate(eval_dataset=test_dataset)
        
#         # Modify results_path to include checkpoint name
#             results_path = os.path.join(training_args.output_dir, "results", checkpoint)
#             os.makedirs(results_path, exist_ok=True)
        
#             with open(os.path.join(results_path, "eval_results.json"), "w") as f:
#                 json.dump(results, f)

In [15]:
#parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
#parser.parse_args_into_dataclasses()

parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments,))
model_args, data_args, training_args, remaining = parser.parse_args_into_dataclasses(return_remaining_strings=True)
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
cache_dir=None,
checkpointing=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=False,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
early_stopping_patience=5,
early_stopping_threshold=0.001,
eval_accumulation_steps=None,
eval_and_save_results=True,
eval_delay=0,
eval_steps=500,
evaluation_strategy=steps,
find_unused_parameters=False,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_bette

In [16]:
if __name__ == "__main__":
    train()

Some weights of the model checkpoint at facebook/opt-125m were not used when initializing OPTForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing OPTForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing OPTForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,model,model.decoder,model.decoder.embed_tokens,model.decoder.embed_positions,model.decoder.final_layer_norm,model.decoder.layers,model.decoder.layers.0,model.decoder.layers.0.self_attn,model.decoder.layers.0.self_attn.k_proj,model.decoder.layers.0.self_attn.v_proj,model.decoder.layers.0.self_attn.q_proj,model.decoder.layers.0.self_attn.out_proj,model.decoder.layers.0.activation_fn,model.decoder.layers.0.self_attn_layer_norm,model.decoder.layers.0.fc1,model.decoder.layers.0.fc2,model.decoder.layers.0.final_layer_norm,model.decoder.layers.1,model.decoder.layers.1.self_attn,model.decoder.layers.1.self_attn.k_proj,model.decoder.layers.1.self_attn.v_proj,model.decoder.layers.1.self_attn.q_proj,model.decoder.layers.1.self_attn.out_proj,model.decoder.layers.1.activation_fn,model.decoder.layers.1.self_attn_layer_norm,model.decoder.layers.1.fc1,model.decoder.layers.1.fc2,model.decoder.layers.1.final_layer_norm,model.decoder.layers.2,model.decoder.layers.2.self_attn,model.decoder.layers.2.self_

Step,Training Loss,Validation Loss,Accuracy,F1,Matthews Correlation,Precision,Recall,Pr Auc,Roc Auc,Brier Score
500,1.601900,1.496253,0.547297,0.472126,0.153880,0.618699,0.549872,0.609154,0.592983,0.253363
1000,1.354700,1.313025,0.579730,0.578046,0.159802,0.580477,0.579329,0.646631,0.625139,0.237999
1500,1.163700,1.065053,0.613514,0.612979,0.227095,0.613805,0.613290,0.704194,0.672865,0.229756
2000,0.802200,0.753646,0.663176,0.662936,0.326399,0.663373,0.663026,0.764065,0.719570,0.217542
2500,0.637900,0.639208,0.643919,0.637363,0.296441,0.653589,0.643040,0.807324,0.754764,0.207966
3000,0.556800,0.528165,0.758277,0.757876,0.519184,0.760601,0.758587,0.853541,0.814722,0.169138
3500,0.434200,0.476471,0.786486,0.781862,0.602362,0.815506,0.787506,0.902066,0.889765,0.150568
4000,0.420800,0.386038,0.837669,0.837582,0.675614,0.838064,0.837551,0.921105,0.916697,0.116961
4500,0.323700,0.390472,0.841723,0.841279,0.688593,0.846488,0.842119,0.928758,0.926934,0.114629
5000,0.333000,0.397082,0.848986,0.848457,0.701609,0.852997,0.848626,0.932629,0.931783,0.112760


/common/zhangz2lab/zhanh/output_budget_0918/results/run
